In [ ]:
!pip install gradio --quiet

import torch
import cv2
import numpy as np
import gradio as gr
from google.colab import drive
from PIL import Image
import os

# Monter Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Chemin du modèle YOLOv5 best.pt sur Drive
MODEL_PATH = "/content/drive/MyDrive/YOLOv5_runs_finetuning2/fine_tuning/updated_model3/weights/best.pt"

# Charger le modèle YOLOv5
detector = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH, force_reload=True)

# Définir des couleurs uniques pour chaque classe
class_colors = {
    'Cap': (255, 0, 0),  # Rouge
    'Defective-Label': (0, 255, 0),  # Vert
    'Good-Label': (0, 0, 255),  # Bleu
    'No-Cap': (255, 255, 0),  # Jaune
    'No-label': (255, 0, 255),  # Magenta
    'bottle': (0, 255, 255)  # Cyan
}

def detect_objects(image, confidence_threshold):
    """Détection d'objets avec affichage des scores sur les bounding boxes."""
    image_cv = np.array(image)
    results = detector(image_cv)

    for *box, conf, cls in results.xyxy[0]:
        if conf < confidence_threshold:
            continue  # Ignorer les détections sous le seuil

        x1, y1, x2, y2 = map(int, box)
        class_name = detector.names[int(cls)]
        color = class_colors.get(class_name, (255, 255, 255))  # Blanc si classe inconnue
        label = f"{class_name} {conf:.2f}"

        # Dessiner la bounding box
        cv2.rectangle(image_cv, (x1, y1), (x2, y2), color, 2)

        # Ajouter un fond pour améliorer la lisibilité du texte
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        cv2.rectangle(image_cv, (x1, y1 - th - 5), (x1 + tw, y1), color, -1)

        # Ajouter le texte de détection avec score
        cv2.putText(image_cv, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return Image.fromarray(image_cv)

# Interface utilisateur avec curseur pour ajuster la confiance
iface = gr.Interface(
    fn=detect_objects,
    inputs=[
        gr.Image(type="pil", label="📷 Uploader une image"),
        gr.Slider(0.0, 1.0, value=0.5, step=0.05, label="🔍 Seuil de Confiance"),
    ],
    outputs=gr.Image(type="pil", label="🔎 Résultat avec Détection"),
    title="Détection de Bouteilles avec YOLOv5",
    description="Uploader une image et ajuster le seuil de confiance pour filtrer les détections.",
)

# Lancer l'interface
iface.launch(share=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-2-25 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e149ac863e7ea57f2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
